In [12]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import scipy
from matplotlib.gridspec import GridSpec
from __future__ import print_function
from mailmerge import MailMerge
import shutil
import openpyxl
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-10-02 14:07:16


In [13]:
Years = range(2005,2017)
Last5Y = range(12,17)
# Roadway and Crash Data
IRISPath = r'C:\Users\mr068144\Downloads\IRIS\IL_IRIS.gdb'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route') for y in Years}
IRIS_Table = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table_HCurve') for y in Years}
CrashPath = r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data\IL_Crash.gdb'
CrashData = {y:CrashPath + '\\CrashExtract_' + str(y) + '_GIS' for y in Years}

Project_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\ILSHSP_Tables'
OutputDir   =  Project_Dir + '\\WorkData\\State_Local_Urban_Rural'
DataDir = Project_Dir + '\\SourceData'
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-10-02 14:07:17


In [14]:
print(strftime("%Y-%m-%d %H:%M:%S"))
def StateLocal(PGN):
    try:
        PGN = str(PGN)
        return({True:'State',False:'Local'}[PGN[0] == 'S'])
    except:
        return('')
def RuralUrban(PGT):
    try:
        PGT = str(PGT)
        return({True:'Urban',False:'Rural'}[PGT.lower().find('urban')!=-1])
    except:
        return('')
def TraffwayConv(TW1,TW2):
    ConvDict = {
        0:'State Rural',
        1:'State Rural',
        2:'State Rural',
        3:'Local Rural',
        4:'State Rural',
        5:'State Urban',
        6:'State Urban',
        8:'Local Urban',
        9:'State Urban'
    }
    tw = ''
    if not math.isnan(TW1):
        tw = int(TW1)
    if not math.isnan(TW2):
        tw = int(TW2)
    if tw in ConvDict.keys():
        return(ConvDict[tw])
def GetCrashTime(y,m,d,s):
    y += 2000
    h = int(s.split(':')[0])
    mm = int(s.split(':')[1].split(' ')[0])
    ampm = s.split(':')[1].split(' ')[1]
    if ampm == 'PM':
            if h<12:
                h+=12
    return(datetime(int(y),int(m),int(d),int(h),int(mm)))
def ConvertLightCondition(l):
    if l in ['Darkness / Lighted Road','Darkness/ Lighted Road']:
        l = 'Darkness, Lighted Road'
    return(l)

2018-10-02 14:07:20


In [15]:
print(strftime("%Y-%m-%d %H:%M:%S"))
#Crash_DF = pd.read_excel(Project_Dir + '\\PreviousFiles\\DataSheets_Crash12to16_KA_v2.xlsx',sheetname='12-16_Crash_KA')
#Crash_DF['KA'] = Crash_DF.Total_killed + Crash_DF.A_injuries
#print(Crash_DF.shape)
Crash_DF = pd.read_excel(Project_Dir + '\\SourceData\\CrashExtract_07to16_KA.xlsx')
Crash_DF['KA'] = Crash_DF.Total_killed + Crash_DF.A_injuries
Crash_DF['Date'] = [GetCrashTime(y,m,d,t) for y,m,d,t in zip(Crash_DF.Crash_Year,Crash_DF.Crash_Month,Crash_DF.Crash_Day,Crash_DF.Time_of_crash)]
Crash_DF = Crash_DF.sort_values('Date')
Crash_DF['LightCondition'] = [ConvertLightCondition(l) for l in Crash_DF.Light_condition]

print(Crash_DF.shape)
Occ_DF = pd.read_excel(Project_Dir + '\\SourceData\\PersonData_07to16_KA.xlsx')
print(Occ_DF.shape)
Veh_DF = pd.read_excel(Project_Dir + '\\SourceData\\VehicleExtract_07to16_KA.xlsx')
Veh_DF.index = Veh_DF.Case_number_CH2M
print(Veh_DF.shape)

EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)

Crash_DF.index = Crash_DF.CaseID_CH2M
Occ_DF.index   = Occ_DF.CaseNum_CH2M
Occ_DF['Pedestrian'] = Occ_DF.Pedestrain
Occ_DF['Crash_Year'] = Occ_DF.Year-2000
Occ_DF['A_injuries'] = [{True:1,False:0}[v==3] for v in Occ_DF['Injury Severity']]
Occ_DF['Total_killed'] = [{True:1,False:0}[v==4] for v in Occ_DF['Injury Severity']]
Occ_DF['KA'] = Occ_DF['Total_killed'] + Occ_DF['A_injuries']
Occ_DF['State_Local_Urban_Rural'] = Crash_DF.State_Local_Urban_Rural.loc[Occ_DF.index]
Occ_DF['District'] = Crash_DF.District.loc[Occ_DF.index]
Occ_DF['Date'] = Crash_DF.Date.loc[Occ_DF.index]
EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)
#for i,r in EADF.iterrows():
#    Occ_DF[r.CrashDataField] = Crash_DF[r.CrashDataField].loc[Occ_DF.index]

2018-10-02 14:07:27
(108250, 111)
(137747, 56)
(177145, 58)


In [16]:
Occ_DF['UnrestrainedOccupants_ISHSP'] = Occ_DF['Unrestrained_Occupants_ISHSP']
for i,r in EADF[EADF.Acronym.isin(['RD','IN','SA','DF','HV','WZ','TN','IM','OD','YD','MC'])].iterrows():
    Occ_DF[r.CrashDataField] = Crash_DF[r.CrashDataField].loc[Occ_DF.index]

In [23]:
EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)
template  = r"\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\ILSHSP_Tables\Template\MergeDocument9.docx"
excelTemp = r"\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\ILSHSP_Tables\Template\Figure_Template3.xlsm"
for i1,r in EADF.iterrows():
    df = Occ_DF[(Occ_DF[r.CrashDataField]=='X') & (Occ_DF.Crash_Year>=12)]
    df = pd.DataFrame(df.groupby(['Crash_Year','State_Local_Urban_Rural'])['A_injuries','Total_killed','KA'].aggregate(np.sum))
    df = df.unstack(['State_Local_Urban_Rural'])
    df = df.fillna(0)
    df = df.astype(int)
    df.loc['Total'] = [sum(df[c]) for c in list(df)]
    
    cdf = pd.DataFrame()
    tot = [df.loc[i,[('Total_killed','State Rural'),('Total_killed','State Urban'),('Total_killed','Local Rural'),('Total_killed','Local Urban')]].sum() for i in list(df.index)]

    cdf['k_year'] = ['2012','2013','2014','2015','2016','Total']

    cdf['k_s_u'] = ['{:,.0f}'.format(v) for v,t in zip(df[('Total_killed','State Urban')],tot)]
    cdf['k_s_r'] = ['{:,.0f}'.format(v) for v,t in zip(df[('Total_killed','State Rural')],tot)]
    cdf['k_s_t'] = ['{:,.0f}'.format(v) for v,t in zip(df[('Total_killed','State Urban')] + df[('Total_killed','State Rural')],tot)]
    cdf['k_s_u_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('Total_killed','State Urban')],tot)]
    cdf['k_s_r_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('Total_killed','State Rural')],tot)]
    cdf['k_s_t_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('Total_killed','State Urban')] + df[('Total_killed','State Rural')],tot)]

    cdf['k_l_u'] = ['{:,.0f}'.format(v) for v,t in zip(df[('Total_killed','Local Urban')],tot)]
    cdf['k_l_r'] = ['{:,.0f}'.format(v) for v,t in zip(df[('Total_killed','Local Rural')],tot)]
    cdf['k_l_t'] = ['{:,.0f}'.format(v) for v,t in zip(df[('Total_killed','Local Urban')] + df[('Total_killed','Local Rural')],tot)]
    cdf['k_l_u_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('Total_killed','Local Urban')],tot)]
    cdf['k_l_r_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('Total_killed','Local Rural')],tot)]
    cdf['k_l_t_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('Total_killed','Local Urban')] + df[('Total_killed','Local Rural')],tot)]

    cdf['k_tot'] = ['{:,.0f}'.format(v) for v,t in zip(df[('Total_killed','Local Urban')] + df[('Total_killed','Local Rural')]+df[('Total_killed','State Urban')] + df[('Total_killed','State Rural')],tot)]
    cdf['k_tot_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('Total_killed','Local Urban')] + df[('Total_killed','Local Rural')]+df[('Total_killed','State Urban')] + df[('Total_killed','State Rural')],tot)]

    tot = [df.loc[i,[('A_injuries','State Rural'),('A_injuries','State Urban'),('A_injuries','Local Rural'),('A_injuries','Local Urban')]].sum() for i in list(df.index)]

    cdf['a_year'] = ['2012','2013','2014','2015','2016','Total']

    cdf['a_s_u'] = ['{:,.0f}'.format(v) for v,t in zip(df[('A_injuries','State Urban')],tot)]
    cdf['a_s_r'] = ['{:,.0f}'.format(v) for v,t in zip(df[('A_injuries','State Rural')],tot)]
    cdf['a_s_t'] = ['{:,.0f}'.format(v) for v,t in zip(df[('A_injuries','State Urban')] + df[('A_injuries','State Rural')],tot)]
    cdf['a_s_u_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('A_injuries','State Urban')],tot)]
    cdf['a_s_r_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('A_injuries','State Rural')],tot)]
    cdf['a_s_t_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('A_injuries','State Urban')] + df[('A_injuries','State Rural')],tot)]

    cdf['a_l_u'] = ['{:,.0f}'.format(v) for v,t in zip(df[('A_injuries','Local Urban')],tot)]
    cdf['a_l_r'] = ['{:,.0f}'.format(v) for v,t in zip(df[('A_injuries','Local Rural')],tot)]
    cdf['a_l_t'] = ['{:,.0f}'.format(v) for v,t in zip(df[('A_injuries','Local Urban')] + df[('A_injuries','Local Rural')],tot)]
    cdf['a_l_u_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('A_injuries','Local Urban')],tot)]
    cdf['a_l_r_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('A_injuries','Local Rural')],tot)]
    cdf['a_l_t_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('A_injuries','Local Urban')] + df[('A_injuries','Local Rural')],tot)]

    cdf['a_tot'] = ['{:,.0f}'.format(v) for v,t in zip(df[('A_injuries','Local Urban')] + df[('A_injuries','Local Rural')]+df[('A_injuries','State Urban')] + df[('A_injuries','State Rural')],tot)]
    cdf['a_tot_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('A_injuries','Local Urban')] + df[('A_injuries','Local Rural')]+df[('A_injuries','State Urban')] + df[('A_injuries','State Rural')],tot)]

    tot = [df.loc[i,[('KA','State Rural'),('KA','State Urban'),('KA','Local Rural'),('KA','Local Urban')]].sum() for i in list(df.index)]
    cdf['ka_year'] = ['2012','2013','2014','2015','2016','Total']

    cdf['ka_s_u'] = ['{:,.0f}'.format(v) for v,t in zip(df[('KA','State Urban')],tot)]
    cdf['ka_s_r'] = ['{:,.0f}'.format(v) for v,t in zip(df[('KA','State Rural')],tot)]
    cdf['ka_s_t'] = ['{:,.0f}'.format(v) for v,t in zip(df[('KA','State Urban')] + df[('KA','State Rural')],tot)]
    cdf['ka_s_u_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('KA','State Urban')],tot)]
    cdf['ka_s_r_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('KA','State Rural')],tot)]
    cdf['ka_s_t_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('KA','State Urban')] + df[('KA','State Rural')],tot)]

    cdf['ka_l_u'] = ['{:,.0f}'.format(v) for v,t in zip(df[('KA','Local Urban')],tot)]
    cdf['ka_l_r'] = ['{:,.0f}'.format(v) for v,t in zip(df[('KA','Local Rural')],tot)]
    cdf['ka_l_t'] = ['{:,.0f}'.format(v) for v,t in zip(df[('KA','Local Urban')] + df[('KA','Local Rural')],tot)]
    cdf['ka_l_u_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('KA','Local Urban')],tot)]
    cdf['ka_l_r_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('KA','Local Rural')],tot)]
    cdf['ka_l_t_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('KA','Local Urban')] + df[('KA','Local Rural')],tot)]

    cdf['ka_tot'] = ['{:,.0f}'.format(v) for v,t in zip(df[('KA','Local Urban')] + df[('KA','Local Rural')]+df[('KA','State Urban')] + df[('KA','State Rural')],tot)]
    cdf['ka_tot_p'] = ['{:.0%}'.format(v/t) for v,t in zip(df[('KA','Local Urban')] + df[('KA','Local Rural')]+df[('KA','State Urban')] + df[('KA','State Rural')],tot)]
   
    
    mdf = pd.DataFrame(columns=['K','A'])
    df = Occ_DF[(Occ_DF[r.CrashDataField]=='X') & (Occ_DF.Crash_Year>=12)]
    
    for i2,r2 in EADF.iterrows():
        edf = df[df[r2.CrashDataField]=='X']
        mdf.loc[r2.Name] = [sum(edf['Total_killed']),sum(edf['A_injuries'])]
    mdf = mdf.sort_values(['K','A'],ascending=False)
    mdf.columns = ['c_ea_k','c_ea_a']
    mdf['c_ea_ka'] = mdf.c_ea_k + mdf.c_ea_a
    mdf['c_ea_k_p']  = ['{:.1%}'.format(float(k)/float(mdf.loc[r.Name,'c_ea_k'])) for k in mdf.c_ea_k] 
    mdf['c_ea_a_p']  = ['{:.1%}'.format(float(a)/float(mdf.loc[r.Name,'c_ea_a'])) for a in mdf.c_ea_a] 
    mdf['c_ea_ka_p'] = ['{:.1%}'.format(float(ka)/float(mdf.loc[r.Name,'c_ea_ka'])) for ka in mdf.c_ea_ka] 
    mdf['c_ea_ka'] = ['{:,.0f}'.format(k) for k in mdf.c_ea_ka] 
    mdf['c_ea_k'] = ['{:,.0f}'.format(k) for k in mdf.c_ea_k] 
    mdf['c_ea_a'] = ['{:,.0f}'.format(k) for k in mdf.c_ea_a] 
    mdf['c_ea'] = list(mdf.index)
    mdf = mdf.astype(str)
    
    document = MailMerge(template)
    document.merge(ea_name=r.Name)
    document.merge(ea_acr=r.Acronym)
    document.merge(ea_def=r.Description)
    L1 = ['k_year','k_s_u','k_s_r','k_s_t','k_l_u','k_l_r','k_l_t','k_tot','k_s_u_p','k_s_r_p','k_s_t_p','k_l_u_p','k_l_r_p','k_l_t_p','k_tot_p']
    D1 = [{v:cdf.loc[y,v] for v in L1} for y in list(cdf.index)]
    L2 = ['a_year','a_s_u','a_s_r','a_s_t','a_l_u','a_l_r','a_l_t','a_tot','a_s_u_p','a_s_r_p','a_s_t_p','a_l_u_p','a_l_r_p','a_l_t_p','a_tot_p']
    D2 = [{v:cdf.loc[y,v] for v in L2} for y in list(cdf.index)]
    L3 = ['c_ea','c_ea_k','c_ea_a','c_ea_ka','c_ea_k_p','c_ea_a_p','c_ea_ka_p']
    D3 = [{v:mdf.loc[y,v] for v in L3} for y in list(mdf.index)]
    #L4 = ['ka_year','ka_s_u','ka_s_r','ka_s_t','ka_l_u','ka_l_r','ka_l_t','ka_tot','ka_s_u_p','ka_s_r_p','ka_s_t_p','ka_l_u_p','ka_l_r_p','ka_l_t_p','ka_tot_p']
    #D4 = [{v:cdf.loc[y,v] for v in L4} for y in list(cdf.index)]
    document.merge_rows('k_year',D1)
    document.merge_rows('a_year',D2)
    document.merge_rows('c_ea',D3)
    #document.merge_rows('ka_year',D4)
    document.write(OutputDir + '\\{}.docx'.format(r.Name.replace('/','_').replace(' ','_')))    
    document.close()
    
    df = Occ_DF[(Occ_DF[r.CrashDataField]=='X')]
    
    pdf = pd.DataFrame(columns=range(2007,2023))
    for year in range(2007,2017):
        ydf = df[df.Crash_Year==year-2000]
        pdf[year] = [ydf.Total_killed.sum(),ydf.A_injuries.sum(),None,None]
    k_ave = np.mean(pdf[[2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]].loc[0])
    a_ave = np.mean(pdf[[2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]].loc[1])
    pdf.loc[2,2016] = k_ave
    pdf.loc[3,2016] = a_ave
    for i,year in enumerate(range(2017,2023)):
        pdf.loc[2,year] = pdf.loc[0,2016] - (i+1)*.02*k_ave 
        pdf.loc[3,year] = pdf.loc[1,2016] - (i+1)*.02*a_ave 
    
    xfile = openpyxl.load_workbook(excelTemp)
    sheet = xfile['Template']
    #sheet.title = ea

    for i,y in enumerate(range(2007,2017)):
        sheet.cell(2,2+i).value = pdf.loc[0,y]
    for i,y in enumerate(range(2007,2017)):
        sheet.cell(3,2+i).value = pdf.loc[1,y]
    for i,y in enumerate(range(2016,2023)):
        sheet.cell(4,11+i).value = pdf.loc[2,y]
    for i,y in enumerate(range(2016,2023)):
        sheet.cell(5,11+i).value = pdf.loc[3,y]
    xfile.save(OutputDir + '\\{}.xlsx'.format(r.Name.replace('/','_').replace(' ','_')))
    xfile.close()

In [21]:
excelTemp = r"\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\ILSHSP_Tables\Template\Figure_Template2.xlsx"
xfile = openpyxl.load_workbook(excelTemp)
sheet = xfile['Template']
for i1,r in EADF.iterrows():
    df = Crash_DF[(Crash_DF.Crash_injury_severity.isin(['A Injury Crash','Fatal Crash'])) & (Crash_DF[r.CrashDataField]=='X')]
    pdf = pd.DataFrame(columns=range(2007,2023))
    for year in range(2007,2017):
        ydf = df[df.Crash_Year==year-2000]
        pdf[year] = [ydf.Total_killed.sum(),ydf.A_injuries.sum(),None,None]
    k_ave = np.mean(pdf[[2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]].loc[0])
    a_ave = np.mean(pdf[[2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]].loc[1])
    pdf.loc[2,2016] = k_ave
    pdf.loc[3,2016] = a_ave
    for i,year in enumerate(range(2017,2023)):
        pdf.loc[2,year] = pdf.loc[0,2016] - (i+1)*.02*k_ave 
        pdf.loc[3,year] = pdf.loc[1,2016] - (i+1)*.02*a_ave 
    
    NewSheet = xfile.copy_worksheet(sheet)
    NewSheet.title = r.Name.replace('/','_').replace(' ','_')

    for i,y in enumerate(range(2007,2017)):
        NewSheet.cell(2,2+i).value = pdf.loc[0,y]
    for i,y in enumerate(range(2007,2017)):
        NewSheet.cell(3,2+i).value = pdf.loc[1,y]
    for i,y in enumerate(range(2016,2023)):
        NewSheet.cell(4,11+i).value = pdf.loc[2,y]
    for i,y in enumerate(range(2016,2023)):
        NewSheet.cell(5,11+i).value = pdf.loc[3,y]
xfile.save(OutputDir + '\\Projections.xlsx')
xfile.close()